In [3]:
import torch
import time
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

class LeNet5(nn.Module):

    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        #x = F.log_softmax(x, dim=1) # 计算log(softmax(x))
        return x

def train_model(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs = 50):
    since = time.time()
  
    best_acc = 0.0
  
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch,num_epochs - 1))
        print('-' * 10)
        model.train() #Set model to training mode

        train_running_loss = 0.0
        train_running_corrects = 0

        test_running_loss = 0.0
        test_running_corrects = 0
        
        for i, data in enumerate(train_dataloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _,preds = torch.max(outputs,1)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            train_running_loss += loss.item() * inputs.size(0)
            train_running_corrects += torch.sum(preds == labels.data)
        
        train_epoch_loss = train_running_loss / len(train_dataloader.dataset)
        train_epoch_acc = train_running_corrects.double() / len(train_dataloader.dataset)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format('train', train_epoch_loss, train_epoch_acc))
        writer.add_scalar('Train/Loss', train_epoch_loss, epoch)
        writer.add_scalar('Train/Acc', train_epoch_acc, epoch)

        model.eval()
        for i, data in enumerate(test_dataloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.no_grad():
                outputs = model(inputs)
                _,preds = torch.max(outputs,1)
                loss = criterion(outputs,labels)

                test_running_loss += loss.item() * inputs.size(0)
                test_running_corrects += torch.sum(preds == labels.data)

        test_epoch_loss = test_running_loss / len(test_dataloader.dataset)
        test_epoch_acc = test_running_corrects.double() / len(test_dataloader.dataset)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format('test', test_epoch_loss, test_epoch_acc))
        writer.add_scalar('Test/Loss', test_epoch_loss, epoch)
        writer.add_scalar('Test/Acc', test_epoch_acc, epoch)
        if test_epoch_acc > best_acc:
            best_acc = test_epoch_acc
    
    print()
    writer.close() 
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60 , time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return model

if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    writer = SummaryWriter('./logs')
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])  # 标准化图像数据

    trainset = datasets.MNIST(root='data', train=True,
                                download=False, transform=transform)
    train_loader = DataLoader(trainset, batch_size=64,
                             shuffle=True)

    testset = datasets.MNIST(root='data', train=False,
                               download=False, transform=transform)
    test_loader = DataLoader(testset, batch_size=32,
                            shuffle=False)
    lr = 0.001
    epochs = 50
    model = LeNet5().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(),lr=1e-3, momentum = 0.9)

    model = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=epochs)

    print("Done!")

Epoch 0/49
----------
train Loss: 1.1429 Acc: 0.6606
test Loss: 0.2871 Acc: 0.9160
Epoch 1/49
----------
train Loss: 0.2072 Acc: 0.9367
test Loss: 0.1326 Acc: 0.9599
Epoch 2/49
----------
train Loss: 0.1272 Acc: 0.9616
test Loss: 0.1102 Acc: 0.9657
Epoch 3/49
----------
train Loss: 0.0972 Acc: 0.9714
test Loss: 0.0768 Acc: 0.9768
Epoch 4/49
----------
train Loss: 0.0820 Acc: 0.9746
test Loss: 0.0695 Acc: 0.9783
Epoch 5/49
----------
train Loss: 0.0717 Acc: 0.9785
test Loss: 0.0587 Acc: 0.9807
Epoch 6/49
----------
train Loss: 0.0639 Acc: 0.9810
test Loss: 0.0556 Acc: 0.9826
Epoch 7/49
----------
train Loss: 0.0577 Acc: 0.9822
test Loss: 0.0528 Acc: 0.9839
Epoch 8/49
----------
train Loss: 0.0526 Acc: 0.9840
test Loss: 0.0457 Acc: 0.9859
Epoch 9/49
----------
train Loss: 0.0490 Acc: 0.9848
test Loss: 0.0467 Acc: 0.9850
Epoch 10/49
----------
train Loss: 0.0456 Acc: 0.9860
test Loss: 0.0424 Acc: 0.9862
Epoch 11/49
----------
train Loss: 0.0426 Acc: 0.9869
test Loss: 0.0465 Acc: 0.9850
Ep